In [1]:
import pandas as pd
import numpy as np
%matplotlib inline

In [2]:
data = pd.read_csv('data/data_update-2017-10.csv')
test = pd.read_csv('data/properati_dataset_testing_noprice.csv')

In [3]:
#test = test[test['state_name'].str.contains('G.B.A') | test['state_name'].str.contains('Capital Federal')]

In [4]:
def descripcion_token(x):
    words = ['parrilla','suite','cochera', 'placard', 'pileta', 'terraza',\
             'vestidor', 'patio', 'hall', 'servicio', 'muebles', 'balcón', 'jardín','aire acondicionado', \
             'quincho', 'fondo', 'piscina','jardin', 'garage', 'parque', 'balcon', 'estrenar', \
             'galería', 'dependencia', 'buena','calefacción','radiante', 'seguridad', \
             'chalet','gimnasio']
    result = ''
    for word in words:
        if word in x:
            result += '|' + word
    return result
def place_name_from_parent(x): ## Segun vi el 3 item del place es generalmente el lugar
    list = x.split('|')
    if(len(list) < 5):
        return np.nan
    else:
        return list[3]
    
def rescatar_barrios_no_oficiales(x):
    if x == 'Once':
        return ("Balvanera")
    elif x == 'Bajo Belgrano':
        return ("Belgrano")
    elif x == 'Alto Palermo':
        return ("Palermo")
    elif x == 'Tribunales':
        return ("San Nicolás")
    elif x == 'Las Cañitas':
        return("Palermo")
    elif x == 'Capital Federal':
        return(np.nan)
    else:
        return x
    
def recuperar_cant_amb(x):
    "Recupera el dato amb en x (puede ser la descripcion o el titulo)"
    try:
        pos = 0
        x = x.lower()
        if "ambientes" in x:
            pos = x.rfind("ambientes")
        elif "amb" in x:
            pos = x.rfind("amb")
        if pos != 0:
            try:
                return (np.float64(x[pos -2]))
            except ValueError:
                if ("dos amb" in x) or ("dos ambientes" in x):
                    return 2.0
                elif ("tres amb" in x) or ("tres ambientes" in x):
                    return 3.0
                elif ("cuatro amb" in x) or ("cuatro ambientes" in x):
                    return 4.0
                elif ("cinco amb" in x) or ("cinco ambientes" in x):
                    return 5.0
                elif ("seis amb" in x) or ("seis ambientes" in x):
                    return 6.0
                return np.nan
    except AttributeError:
        return None

In [5]:
data.loc[:,'place_name'] = data.place_with_parent_names.apply(place_name_from_parent) ## esto quita muchos problemas 
                                                                                    ## de place name como nombres de country
                                                                                    ## o palermo soho
data.loc[data.description.notnull(),'description'] = data.loc[data.description.notnull(),'description'].apply(descripcion_token)
#test.loc[:,'place_name'] = test.place_with_parent_names.apply(place_name_from_parent)

In [6]:
data.loc[:,'place_name'] = data.place_name.apply(rescatar_barrios_no_oficiales)
#test.loc[:,'place_name'] = test.place_name.apply(rescatar_barrios_no_oficiales)

In [7]:
data.dropna(axis=0,subset=['price_aprox_usd'],inplace=True) ## drop de los datos sin precio
data = data.loc[data['price_aprox_usd'] > 0,:]

In [8]:
data.loc[data.rooms.isnull(),'rooms'] = data.description.apply(recuperar_cant_amb)
data.loc[data.rooms.isnull(),'rooms'] = data.title.apply(recuperar_cant_amb)
data.loc[data.surface_total_in_m2.isnull(),'surface_total_in_m2'] = data.surface_covered_in_m2
#test.loc[test.rooms.isnull(),'rooms'] = test.description.apply(recuperar_cant_amb)

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 284732 entries, 0 to 317037
Data columns (total 23 columns):
id                            284732 non-null object
created_on                    284732 non-null object
property_type                 284732 non-null object
place_name                    278472 non-null object
place_with_parent_names       284732 non-null object
state_name                    284732 non-null object
geonames_id                   224194 non-null float64
lat-lon                       202676 non-null object
lat                           202676 non-null float64
lon                           202676 non-null float64
price                         284732 non-null float64
currency                      284732 non-null object
price_aprox_local_currency    284732 non-null float64
price_aprox_usd               284732 non-null float64
surface_total_in_m2           266756 non-null float64
surface_covered_in_m2         257298 non-null float64
price_usd_per_m2              201

In [10]:
data.describe()

,geonames_id,lat,lon,price,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses
count,2.241940e+05,202676.000000,202676.000000,2.847320e+05,2.847320e+05,2.847320e+05,2.667560e+05,2.572980e+05,2.012090e+05,2.541550e+05,36551.000000,168778.000000,3.620700e+04
mean,3.498000e+06,-34.557248,-58.525949,3.346441e+05,4.267056e+06,2.545583e+05,7.756810e+03,7.948382e+03,2.001651e+03,4.427643e+03,14.513119,2.931780,3.695044e+03
std,4.424446e+05,1.376284,1.120437,7.408342e+05,6.123039e+06,3.653688e+05,3.872343e+06,3.942872e+06,4.586174e+03,2.073286e+04,105.542985,1.461706,7.465248e+04
min,3.427208e+06,-51.626650,-122.419415,2.600000e+03,4.757708e+04,2.754180e+03,0.000000e+00,0.000000e+00,8.000000e-02,5.000000e-05,1.000000,0.000000,1.000000e+00
25%,3.429595e+06,-34.629218,-58.579659,1.050000e+05,1.660724e+06,9.900000e+04,4.800000e+01,4.600000e+01,1.120690e+03,1.527778e+03,2.000000,2.000000,1.000000e+03
50%,3.430787e+06,-34.592540,-58.481101,1.720000e+05,2.683515e+06,1.600000e+05,8.300000e+01,7.700000e+01,1.785714e+03,2.095532e+03,3.000000,3.000000,1.900000e+03
75%,3.435719e+06,-34.518473,-58.417314,3.270000e+05,4.808730e+06,2.890000e+05,2.000000e+02,1.600000e+02,2.450000e+03,2.896774e+03,6.000000,4.000000,3.800000e+03
max,6.693230e+06,52.619706,58.496000,1.234568e+08,7.984250e+08,5.000000e+07,2.000000e+09,2.000000e+09,1.271027e+06,4.000000e+06,7071.000000,40.000000,1.000150e+07


In [11]:
#Filtrar datos anomalos

In [12]:
# Nan para los datos faltantes es mejor que dropear todo el dato 
# Los nros estan masomenos pensados MASOMENOS JEJE
data.loc[(data['surface_total_in_m2']>5000) | (data['surface_total_in_m2'] < 30),'surface_total_in_m2'] = np.nan
data.loc[(data['surface_covered_in_m2']>5000) | (data['surface_covered_in_m2'] < 30),'surface_covered_in_m2'] = np.nan
data.loc[data['floor']>30,'floor'] = np.nan
data.loc[data['rooms']==0,'rooms'] = np.nan
data.loc[data['rooms']>10,'rooms'] = np.nan
data.loc[(data['lat']>-33) | (data['lat']<-36),['lat','lon','lat-lon']] = np.nan
data.loc[(data['lon']>-57) | (data['lon']<-60),['lat','lon','lat-lon']] = np.nan
data.loc[(data['expenses']>14000) | (data['expenses']<100),'expenses'] = np.nan

In [13]:
data.describe()

,geonames_id,lat,lon,price,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses
count,2.241940e+05,201991.000000,201991.000000,2.847320e+05,2.847320e+05,2.847320e+05,246119.000000,242966.000000,2.012090e+05,2.541550e+05,35771.000000,168593.000000,33221.000000
mean,3.498000e+06,-34.580672,-58.504874,3.346441e+05,4.267056e+06,2.545583e+05,203.475530,131.712705,2.001651e+03,4.427643e+03,4.437310,2.920525,2868.053972
std,4.424446e+05,0.111066,0.151545,7.408342e+05,6.123039e+06,3.653688e+05,319.993164,162.130745,4.586174e+03,2.073286e+04,3.848211,1.409492,2580.415831
min,3.427208e+06,-35.765171,-59.664010,2.600000e+03,4.757708e+04,2.754180e+03,30.000000,30.000000,8.000000e-02,5.000000e-05,1.000000,1.000000,100.000000
25%,3.429595e+06,-34.628929,-58.579640,1.050000e+05,1.660724e+06,9.900000e+04,54.000000,50.000000,1.120690e+03,1.527778e+03,2.000000,2.000000,1100.000000
50%,3.430787e+06,-34.592502,-58.480858,1.720000e+05,2.683515e+06,1.600000e+05,93.000000,81.000000,1.785714e+03,2.095532e+03,3.000000,3.000000,2000.000000
75%,3.435719e+06,-34.518706,-58.417173,3.270000e+05,4.808730e+06,2.890000e+05,215.000000,170.000000,2.450000e+03,2.896774e+03,6.000000,4.000000,3900.000000
max,6.693230e+06,-33.677756,-57.848496,1.234568e+08,7.984250e+08,5.000000e+07,5000.000000,5000.000000,1.271027e+06,4.000000e+06,30.000000,10.000000,14000.000000


In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 284732 entries, 0 to 317037
Data columns (total 23 columns):
id                            284732 non-null object
created_on                    284732 non-null object
property_type                 284732 non-null object
place_name                    278472 non-null object
place_with_parent_names       284732 non-null object
state_name                    284732 non-null object
geonames_id                   224194 non-null float64
lat-lon                       201991 non-null object
lat                           201991 non-null float64
lon                           201991 non-null float64
price                         284732 non-null float64
currency                      284732 non-null object
price_aprox_local_currency    284732 non-null float64
price_aprox_usd               284732 non-null float64
surface_total_in_m2           246119 non-null float64
surface_covered_in_m2         242966 non-null float64
price_usd_per_m2              201

In [15]:
# Dropeo las columnas que no se usan porque el test no las tiene o no son necesarias
data.drop(['price','price_usd_per_m2','price_per_m2','currency',\
          'id','created_on','geonames_id','lat-lon','price_aprox_local_currency',\
         'title','place_with_parent_names'], axis=1, inplace=True)
#test.drop(['id','created_on','lat-lon','description','country_name'], axis=1, inplace=True)
#datos con mas de 4 nan los dropeo el 4 nro medio random
data.dropna(thresh=8,inplace=True)

In [16]:
#Esto es importante, convierto los datos categoricos en numericos cada categoria una columna con 1 si true o 0 si false
##https://stackoverflow.com/questions/34265102/xgboost-categorical-variables-dummification-vs-encoding
##https://machinelearningmastery.com/data-preparation-gradient-boosting-xgboost-python/
data = data.join(pd.get_dummies(data.property_type))
data.drop('property_type',axis=1, inplace=True)
data = data.join(data.description.str.get_dummies())
data.drop('description',axis=1, inplace=True)
data = data.join(pd.get_dummies(data.place_name))
data.drop('place_name',axis=1, inplace=True)
data = data.join(pd.get_dummies(data.state_name))
data.drop('state_name',axis=1, inplace=True)

In [17]:
#####XGBOOST
#https://github.com/dmlc/xgboost
#https://xgboost.readthedocs.io/en/latest/build.html
#https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/
#https://www.slideshare.net/ShangxuanZhang/kaggle-winning-solution-xgboost-algorithm-let-us-learn-from-its-author


In [31]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14166 entries, 0 to 14165
Data columns (total 17 columns):
id                         14166 non-null int64
created_on                 14166 non-null object
property_type              14166 non-null object
operation                  14166 non-null object
place_name                 14166 non-null object
place_with_parent_names    14166 non-null object
country_name               14166 non-null object
state_name                 14166 non-null object
lat-lon                    10487 non-null object
lat                        10487 non-null float64
lon                        10487 non-null float64
surface_total_in_m2        11853 non-null float64
surface_covered_in_m2      13005 non-null float64
floor                      1368 non-null float64
rooms                      7500 non-null float64
expenses                   2543 non-null object
description                14166 non-null object
dtypes: float64(6), int64(1), object(10)
memory usage: 1

In [36]:
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from xgboost import XGBRegressor
import scipy.stats as st

from sklearn.metrics import mean_squared_error

In [19]:
#divide el set de datos en parte de entrenamiento y parte de test
# no se puede usar el test bajado de kaggle porque no tiene el precio
X = data.drop(["price_aprox_usd"], axis=1)
Y = data["price_aprox_usd"]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,test_size=0.33)

In [32]:
## Random search de hiperparametros y entrenamiento del algoritmo, 
## cuando termina tira dos scores (no se cual es mejor) y los mejores hiperparametros que dieron esos scores
## ES LA PARTE QUE TARDA
#http://danielhnyk.cz/how-to-use-xgboost-in-python/
one_to_left = st.beta(10, 1)  
from_zero_positive = st.expon(0, 50)

params = {  
    "n_estimators": st.randint(180,200),#nros random desde a hasta b
    "max_depth": st.randint(12, 25),
    "learning_rate": st.uniform(0.052, 0.003), # nros random de una distribucion uniforme entre a y a+b
    "colsample_bytree": st.uniform(0.97,0.03), 
    "subsample": st.uniform(0.95,0.05), 
    "gamma": st.uniform(9, 1),
    'reg_alpha': [20,30,80,100],
    "min_child_weight": [10,15,20,30]
}

xgb = XGBRegressor(nthread=2) 

grid = RandomizedSearchCV(xgb, params, n_jobs=1,n_iter=5,verbose=10,cv=5) # n_iters es la cant de veces que busca, 10 es lo default
grid.fit(X, Y, eval_metric="rmse")


Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV] reg_alpha=0.1, max_depth=18, gamma=9.46447461657, subsample=0.950015253751, min_child_weight=45, n_estimators=198, learning_rate=0.0522935382359, colsample_bytree=0.985795919378 
[CV]  reg_alpha=0.1, max_depth=18, gamma=9.46447461657, subsample=0.950015253751, min_child_weight=45, n_estimators=198, learning_rate=0.0522935382359, colsample_bytree=0.985795919378, score=0.4195929323828457, total= 8.8min
[CV] reg_alpha=0.1, max_depth=18, gamma=9.46447461657, subsample=0.950015253751, min_child_weight=45, n_estimators=198, learning_rate=0.0522935382359, colsample_bytree=0.985795919378 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  8.9min remaining:    0.0s


[CV]  reg_alpha=0.1, max_depth=18, gamma=9.46447461657, subsample=0.950015253751, min_child_weight=45, n_estimators=198, learning_rate=0.0522935382359, colsample_bytree=0.985795919378, score=0.7551632917467925, total= 8.7min
[CV] reg_alpha=0.1, max_depth=18, gamma=9.46447461657, subsample=0.950015253751, min_child_weight=45, n_estimators=198, learning_rate=0.0522935382359, colsample_bytree=0.985795919378 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 17.8min remaining:    0.0s


[CV]  reg_alpha=0.1, max_depth=18, gamma=9.46447461657, subsample=0.950015253751, min_child_weight=45, n_estimators=198, learning_rate=0.0522935382359, colsample_bytree=0.985795919378, score=0.4494894656398375, total= 8.8min
[CV] reg_alpha=0.1, max_depth=18, gamma=9.46447461657, subsample=0.950015253751, min_child_weight=45, n_estimators=198, learning_rate=0.0522935382359, colsample_bytree=0.985795919378 


...

[CV]  reg_alpha=100, max_depth=16, gamma=9.82912959629, subsample=0.991403166607, min_child_weight=15, n_estimators=197, learning_rate=0.0546410210312, colsample_bytree=0.979034684104, score=0.6854238403614852, total= 7.6min
[CV] reg_alpha=100, max_depth=16, gamma=9.82912959629, subsample=0.991403166607, min_child_weight=15, n_estimators=197, learning_rate=0.0546410210312, colsample_bytree=0.979034684104 


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 75.8min remaining:    0.0s


[CV]  reg_alpha=100, max_depth=16, gamma=9.82912959629, subsample=0.991403166607, min_child_weight=15, n_estimators=197, learning_rate=0.0546410210312, colsample_bytree=0.979034684104, score=0.6795280508356119, total= 7.6min
[CV] reg_alpha=1e-05, max_depth=24, gamma=9.66233219293, subsample=0.975015897969, min_child_weight=45, n_estimators=182, learning_rate=0.0537923819452, colsample_bytree=0.994877696361 
[CV]  reg_alpha=1e-05, max_depth=24, gamma=9.66233219293, subsample=0.975015897969, min_child_weight=45, n_estimators=182, learning_rate=0.0537923819452, colsample_bytree=0.994877696361, score=0.4196993565973991, total=11.9min
[CV] reg_alpha=1e-05, max_depth=24, gamma=9.66233219293, subsample=0.975015897969, min_child_weight=45, n_estimators=182, learning_rate=0.0537923819452, colsample_bytree=0.994877696361 
[CV]  reg_alpha=1e-05, max_depth=24, gamma=9.66233219293, subsample=0.975015897969, min_child_weight=45, n_estimators=182, learning_rate=0.0537923819452, colsample_bytree=0.994

[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed: 231.8min finished


RandomizedSearchCV(cv=5, error_score='raise',
          estimator=XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=2,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1),
          fit_params=None, iid=True, n_iter=5, n_jobs=1,
          param_distributions={'reg_alpha': [1e-05, 0.01, 0.1, 1, 100], 'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f046c32a198>, 'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f046c32ae10>, 'subsample': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f...32a2b0>, 'colsample_bytree': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f046c32aa58>},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score=True, scoring=None, verbose=10)

In [34]:
print(grid.best_score_)
#print(grid.best_estimator_)
print(grid.best_params_)
#print(grid.cv_results_)

0.600642649108
{'reg_alpha': 100, 'max_depth': 16, 'gamma': 9.829129596285604, 'subsample': 0.99140316660671524, 'min_child_weight': 15, 'n_estimators': 197, 'learning_rate': 0.054641021031217096, 'colsample_bytree': 0.97903468410439032}


In [ ]:
# Parte para probar los parametros sin random search
# no tarda mucho porque tira el score de la combinacion de parametros que le pones nada mas
xgb = XGBRegressor(colsample_bytree=0.98,gamma=9.83,learning_rate=0.055,max_depth=16,min_child_weight=15,n_estimators=200,\
                  reg_alpha=100,subsample=0.99,nthread=2) 

xgb.fit(X_train, Y_train,eval_metric="rmse")


In [38]:
# Print the rmse score
print(mean_squared_error(Y_test, xgb.predict(X_test)))

42773361988.7


In [25]:
 df1 = pd.DataFrame({'A': ['A|B|C', 'A|B|C', 'A|B|C', 'A|B|C'],'B': ['A|B|C', 'A|B|C', 'A|B|C', 'A|B|C']},index=[0, 1, 2, 3])

In [27]:
df1.A.str.get_dummies()

,A,B,C
0,1,1,1
1,1,1,1
2,1,1,1
3,1,1,1
